### facial_recognition needs dlib and cmake

#### dlib 
It's a landmark's facial detector with pre-trained models, the dlib is used to estimate the location of 68 coordinates (x, y) that map the facial points on a person's face
dlib is a toolkit for making real world machine learning and data analysis applications in C++. While the library is originally written in C++, it has good, easy to use Python bindings.

#### cmake
CMake is an open-source, cross-platform family of tools designed to build, test and package software. CMake is used to control the software compilation process using simple platform and compiler independent configuration files, and generate native makefiles and workspaces that can be used in the compiler environment of your choice. The suite of CMake tools were created by Kitware in response to the need for a powerful, cross-platform build environment for open-source projects such as ITK and VTK.

#### Face Recognition
Recognize and manipulate faces from Python or from the command line with
the world’s simplest face recognition library.
Built using dlib’s state-of-the-art face recognition built with deep learning. 
The model has an accuracy of 99.38% on the
Labeled Faces in the Wild benchmark.

https://pypi.org/project/face-recognition/

https://face-recognition.readthedocs.io/en/latest/face_recognition.html

https://face-recognition.readthedocs.io/en/latest/readme.html

https://pythonprogramming.net/facial-recognition-python/

#### RGB
https://www.rapidtables.com/web/color/RGB_Color.html

#### RGB to Gray
https://web.stanford.edu/class/cs101/image-6-grayscale-adva.html#:~:text=The%20RGB%20scale%20is%20calibrated,%2C%20green%2C%20or%20blue%20hue.

https://www.geeksforgeeks.org/matlab-rgb-image-to-grayscale-image-conversion/

In [1]:
import cv2 # draw rectangle on face and label image
import os
import face_recognition
import matplotlib.pyplot as plt
import pickle
import time

In [2]:
KNOWN_FACES_DIR = "known_faces2"
# UNKNOWN_FACES_DIR = "unknown_faces"
TOLERANCE = 0.6 # the lower the tolerance the less chance you have to get false positive
# the more is the tolerance the more there will be matching and it can be more risk of incorrect matching.
# if tolerance is less then matching will be mostly correct and there can be less mtching and more flase negative

FRAME_THICKNESS = 3 # size of frame around face. It depends on how big your image is.
FONT_THICKNESS = 2
MODEL="hog" # or we can use "cnn" model of we use gpu because "ccn" is slow in cpu

video = cv2.VideoCapture(0)

In [3]:
known_faces = []
known_names = []

for name in os.listdir(KNOWN_FACES_DIR):
    
    f=None
    # Next we load every file of faces of known person
    for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'):
        if f==None:
            print(f"No of {name} photos = {len(os.listdir(f'{KNOWN_FACES_DIR}/{name}'))}")
            f=1

        # Load an image
#         image = face_recognition.load_image_file(f'{KNOWN_FACES_DIR}/{name}/{filename}')
#         encoding = face_recognition.face_encodings(image)[0]
        encoding = pickle.load(open(f"{KNOWN_FACES_DIR}/{name}/{filename}", "rb"))

        # Append encodings and name
        known_faces.append(encoding)
        known_names.append(int(name))

print(known_names)
if len(known_names)>0:
    next_id = max(known_names) + 1
else:
    next_id = 0

No of 0 photos = 1
No of 1 photos = 1
No of 2 photos = 1
No of 3 photos = 1
[0, 1, 2, 3]


In [4]:
print("processing unknown faces")

processing unknown faces


In [5]:
while True:
    
    ret, image = video.read()
    
    locations = face_recognition.face_locations(image, model=MODEL) # A list of tuples of found face locations in css (top, right, bottom, left) order

    encodings = face_recognition.face_encodings(image, locations)
    
    if locations==[]:
        cv2.putText(image, "No Face is detected in this image", (0 + 10, 0 + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), FONT_THICKNESS)

    for face_encoding, face_location in zip(encodings, locations):

        results = face_recognition.compare_faces(known_faces, face_encoding, TOLERANCE)

        match = None
        if True in results:
            match = known_names[results.index(True)]
            print(f' - {match} from {results}')
        else:
            match = str(next_id)
            next_id+=1
            known_names.append(match)
            known_faces.append(face_encoding)
            os.mkdir(f"{KNOWN_FACES_DIR}/{match}")
            pickle.dump(face_encoding, open(f"{KNOWN_FACES_DIR}/{match}/{match}--{int(time.time())}.pkl", "wb"))
            

        # Each location contains positions in order: top, right, bottom, left
        top_left = (face_location[3], face_location[0])
        bottom_right = (face_location[1], face_location[2])

        color = [0, 255, 0] # BGR

        cv2.rectangle(image, top_left, bottom_right, color, FRAME_THICKNESS)

        top_left = (face_location[3], face_location[2])
        bottom_right = (face_location[1], face_location[2] + 22)

        cv2.rectangle(image, top_left, bottom_right, color, cv2.FILLED)

        cv2.putText(image, str(match), (face_location[3] + 10, face_location[2] + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (43,37,9), FONT_THICKNESS)
#         else:
#             print(f"Face is not recognised")
#             # Each location contains positions in order: top, right, bottom, left [x-axis = left,right and y-axis=bottom,top]
#             top_left = (face_location[3], face_location[0])
#             bottom_right = (face_location[1], face_location[2])

#             color = [0, 0, 255] # BGR

#             cv2.rectangle(image, top_left, bottom_right, color, FRAME_THICKNESS)

#             top_left = (face_location[3], face_location[2])
#             bottom_right = (face_location[1], face_location[2] + 22)

#             cv2.rectangle(image, top_left, bottom_right, color, cv2.FILLED)

#             cv2.putText(image, "UNKNOWN", (face_location[3] + 10, face_location[2] + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (43,37,9), FONT_THICKNESS)


    cv2.imshow('frame', image)
    if cv2.waitKey(1) == ord('q'):
        break
#     cv2.waitKey(10000)
#     cv2.destroyWindow(filename)

video.release()
cv2.destroyAllWindows()

 - 0 from [True, False, False, False, False]
 - 0 from [True, False, False, False, False]
 - 0 from [True, False, False, False, False]
 - 1 from [False, True, False, False, True]
 - 0 from [True, False, False, False, False]
 - 0 from [True, False, False, False, False]
 - 0 from [True, False, False, False, False]
 - 0 from [True, False, False, False, False]
 - 0 from [True, False, False, True, False]
